## An Object-Oriented Chatbot using GPT and a Database
This allows multiple chatbot types (e.g. a health coach and a learning assistant) to be created. Multiple chatbot instances can be created per chatbot type (e.g. for User X and User Y). Both, type and instance are stored and referenced with an ID (e.g. with a UUID) in the database.

This can support the deployment of chatbots in a web backend (state-less). For example, the UUIDs of the type and instance can be read as URL parameters from a URL that users have received from you.

A chatbot is created with the following arguments.
- database_file: File of SQLite (in Folder data/)
- type_id: Reference to chatbot type
- instance_id: Reference to chatbot instance (typically one per user - however, may also be shared by multiple users)
- type_role: Role of chatbot type (will be turned into a first prompt with role:system)
- instance_context: Context of chatbot instance (will be turned into a second prompt with role:system)
- instance_starter: Will be used to generate an initial message to the user (will be turned into a final prompt with role:system)

The following functions are meant to be used from an application (e.g. from controllers of a REST API).
- conversation_retrieve(with_system=False): Retrieve the previous conversation history (default: without prompts with role:system)
- starter(): Returns an initial message to the user (Resulting from instance_starter prompt)
- response_for(user_says): Returns an assistance response to user_says

In [2]:
import uuid
import time
from chatbot_db import Chatbot

#### Create a chatbot "Coach" for user X

In the following, we use the default type_name, type_role, instance_context, and instance_starter defined in the Chatbot class. Provide your own prompts instead.

In [3]:
# bot = Chatbot(
#     database_file="data/chatbot.db", 
#     type_id="9DB9EC22-71E6-4A10-B454-DCBF77CA109B",
#     user_id="68F5F770-3788-4D3C-8F4A-9B229EE11AD0",
#     type_name=Chatbot.default_type_name,
#     type_role=Chatbot.default_type_role,
#     instance_context=Chatbot.default_instance_context,
#     instance_starter=Chatbot.default_instance_starter
# )

type_name = "Act as a Linux Terminal"
type_role = "I want you to act as a linux terminal."
instance_context = "I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. When I need to tell you something in English, I will do so by putting text inside curly brackets {like this}."
instance_starter = "My first command is pwd"

bot = Chatbot(
    database_file="data/chatbot.db", 
    type_id="9DB9EC22-71E6-4A10-B454-DCBF77CA109B",
    user_id="68F5F770-3788-4D3C-8F4A-9B229EE11AD0",
    type_name=type_name,
    type_role=type_role,
    instance_context=instance_context,
    instance_starter=instance_starter
)

Retrieve the complete conversation (held so far) or the bot information (type_role, instance_context, and instance_starter) as retrieved from the database. Both may be used to display that on a frontend.

In [4]:
bot = Chatbot(
    database_file="data/chatbot.db", 
    type_id="9DB9EC22-71E6-4A10-B454-DCBF77CA109B",
    user_id="68F5F770-3788-4D3C-8F4A-9B229EE11AD0"
)
print(bot.conversation_retrieve(with_system=True))
print(bot.info_retrieve())

[{'role': 'system', 'content': 'I want you to act as a linux terminal.'}, {'role': 'system', 'content': 'I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. When I need to tell you something in English, I will do so by putting text inside curly brackets {like this}.'}]
{'name': 'Act as a Linux Terminal', 'role': 'I want you to act as a linux terminal.', 'context': 'I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. When I need to tell you something in English, I will do so by putting text inside curly brackets {like this}.'}


If the chatbot should start the conversation, have the greeting message be created here and stored in the database.

In [5]:
print(bot.starter())

/home/user


URL to be handed out to the user

##### Generic URL
https://[your pythonanywhere user name].pythonanywhere.com/[type id]/[user_id]/chat

https://raaafael.pythonanywhere.com/9DB9EC22-71E6-4A10-B454-DCBF77CA109B/68F5F770-3788-4D3C-8F4A-9B229EE11AD0/chat

##### For Example
https://monkey23.pythonanywhere.com/053e97a0-6a91-4589-8602-340aa47b6376/7515865e-4097-4dd7-9567-d3c7a4c1ed07/chat

#### Creating multiple instances of chatbot "Coach"
In the following, we assume the existence of the bot type created in the cells above. We show example code that will generate N bot instances of that type. Each instance has it's own prompts that will be appended to the type prompts.

In [6]:
# Amount instances to be created
number_of_instances = 5

# Change to list with hardcoded value, e.g., if you want to use existing users.
user_ids = [str(uuid.uuid4()) for _ in range(number_of_instances)]

c  = 0 # counter for successful requests, don't change
error_c = 0 # counter for failed requests, don't change
for user_id in user_ids:
    bot = Chatbot(
        database_file="data/chatbot.db", 
        type_id="9DB9EC22-71E6-4A10-B454-DCBF77CA109B",
        user_id=user_id,
        instance_context=instance_context,
        instance_starter=instance_starter
    )
    try:
        # each bot should have a first message to the user
        print(bot.starter())
    except:
        error_c += 1
        continue
    c+=1
    time.sleep(15) #openai seems to produce more errors if we send the requests too fast.
    
print("successful: {}, failed: {}".format(c, error_c))


`/home/user`
/home/user
```bash
/home/user
```
/home/user
/home/user
successful: 5, failed: 0


#### Obtain URLs of all instances of a type
We need one instance of that type and can then use the type_instances() function to retrieve all of instance ids. Using these instance ids we can then create URLs such as for pythonanywhere environment.

In [7]:
pythonanywhere_username = "raaafael"
type_id = "9DB9EC22-71E6-4A10-B454-DCBF77CA109B"
bot = Chatbot(
    database_file="data/chatbot.db", 
    type_id=type_id,
    user_id=user_ids[0]
)

for user_id in bot.type_instances():
    print("https://{}.pythonanywhere.com/{}/{}/chat".format(pythonanywhere_username, type_id, user_id))

https://raaafael.pythonanywhere.com/9DB9EC22-71E6-4A10-B454-DCBF77CA109B/020294f2-e814-427b-82a5-59e29f256e52/chat
https://raaafael.pythonanywhere.com/9DB9EC22-71E6-4A10-B454-DCBF77CA109B/0d795603-5719-4275-94f6-bb45620e318a/chat
https://raaafael.pythonanywhere.com/9DB9EC22-71E6-4A10-B454-DCBF77CA109B/403f64e8-0ebb-4d90-be19-329765a606ac/chat
https://raaafael.pythonanywhere.com/9DB9EC22-71E6-4A10-B454-DCBF77CA109B/68F5F770-3788-4D3C-8F4A-9B229EE11AD0/chat
https://raaafael.pythonanywhere.com/9DB9EC22-71E6-4A10-B454-DCBF77CA109B/a14cbdd2-d1fc-44c4-b035-1cd2fd238138/chat
https://raaafael.pythonanywhere.com/9DB9EC22-71E6-4A10-B454-DCBF77CA109B/caa7f1f4-7101-4eaa-864b-50814227e953/chat
